# Import Functions

In [ ]:
!nvidia-smi

In [2]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="3"
from pathlib import Path
import numpy as np
#import hyperspy.api as hs
import matplotlib.pyplot as plt
#from dask_ml.preprocessing import StandardScaler
from sklearn.preprocessing import StandardScaler
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader
import numpy as np 
import matplotlib.pyplot as plt 
from torch.autograd import Function
from collections import OrderedDict
import torch.nn as nn
import math
import pickle
import torch.autograd as autograd
import matplotlib.gridspec as gridspec
import os
import h5py
# from torchsummary import summary
# import pytorch_model_summary as pms
from tqdm import tqdm
#import torch.tensor as tensor
from scipy import ndimage
import matplotlib.pyplot as plt
import skimage
from skimage.feature import peak_local_max
from skimage import data, img_as_float,feature
from skimage import io
import cv2

In [3]:
import matplotlib.pylab as pylab

params = {'axes.titlesize':20,
          'xtick.direction': 'in' ,
          'ytick.direction' : 'in',
          'xtick.top' : True,
          'ytick.right' : True,
          'ytick.labelsize':16,
          'xtick.labelsize':16
         }

pylab.rcParams.update(params)

In [ ]:
torch.__version__

In [5]:
#import atomai as aoi
#import kornia as K
import cv2
import scipy
import argparse
import skimage
from skimage.util import random_noise
from skimage import feature
import glob
from scipy import ndimage
import scipy as sp
import random

In [6]:
import warnings
warnings.filterwarnings('ignore') 

In [ ]:
from functools import partial
import numpy as np
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import random_split
from ray import tune
from ray.tune import CLIReporter
from ray.tune.schedulers import ASHAScheduler
from ray.air import session

In [8]:
# import sys
# !{sys.executable} -m pip install -U "ray[tune]"

In [ ]:
torch.cuda.device_count()

# Build two type of mask for two training process

## Set the mask function

# Loading Data

# get dataset from: https://zenodo.org/records/10836435

In [10]:
# Set data direction
data_dir = os.path.abspath("Extremely_Noisy_4DSTEM_Strain_Mapping_Using_CC_ST_AE_Simulated/polycrystal_output4D.mat")

## Load data function 1 for first training process

## Load label

In [11]:
label_xx = np.load('Label_strain_xx.npy').reshape(-1)
label_yy = np.load('Label_strain_yy.npy').reshape(-1)
label_xy = np.load('Label_shear_xy.npy').reshape(-1)
label_rotation = np.load('Label_rotation.npy').reshape(-1)

In [12]:
supervised_label = np.stack((label_xx,label_yy,label_xy,label_rotation),axis=1)

In [ ]:
supervised_label.shape

In [ ]:
fig,ax = plt.subplots(1,4,figsize=(16,4))
ax[0].hist(supervised_label[:,0],200,range=[-0.03,0.03]);
ax[1].hist(supervised_label[:,1],200,range=[-0.03,0.03]);
ax[2].hist(supervised_label[:,2],200,range=[-0.03,0.03]);
ax[3].hist(supervised_label[:,3],200,range=[2,3.2]);

In [15]:
def load_data_4_process2(data_dir, label,w_bg=0.60):
    
    '''
    
        data_dir: path of the dataset
        label_index: path of the pretrained rotation 
    
    '''
    
    f = h5py.File(data_dir,'r')
    op4d = f['output4D']
    op4d = op4d[:,:,28:228,28:228]
    op4d = np.transpose(op4d, (1, 0, 3, 2))
    op4d = op4d.reshape(-1,200,200)
    f.close()
    
    if w_bg == 0:
        
        noisy_data = op4d*1e5/4
    
    else:
    
        noisy_data = np.zeros([65536,200,200])
        im=np.zeros([200,200])
        counts_per_probe = 1e5
        for i in tqdm(range(65536),leave=True,total=65536):
            test_img = np.copy(op4d[i])
            qx = np.fft.fftfreq( im.shape[0], d = 1)
            qy = np.fft.fftfreq( im.shape[1], d = 1)
            qya, qxa = np.meshgrid(qy, qx)
            qxa = np.fft.fftshift(qxa)
            qya = np.fft.fftshift(qya) 
            qra2 = qxa**2 + qya**2
            im_bg = 1./( 1 + qra2 / 1e-2**2 )
            im_bg = im_bg / np.sum(im_bg) 
            int_comb = test_img * (1 - w_bg) + im_bg * w_bg 
            int_noisy = np.random.poisson(int_comb * counts_per_probe) / counts_per_probe
            int_noisy = int_noisy*1e5/4
            noisy_data[i] = int_noisy
        
    del op4d
    
    noisy_data = noisy_data.reshape(-1,1,200,200)
    if len(noisy_data) != len(label):
        return('data and label do not pair')
#     data_ = Dataset(noisy_data, label)
    data_with_label = []
    print('add image-rotation pair to whole dataset')
    for i in tqdm(range(noisy_data.shape[0]), leave=True, total=noisy_data.shape[0]):
        data_with_label.append([noisy_data[i], label[i]])
    del(noisy_data)
    
    return data_with_label

In [ ]:
data_ = load_data_4_process2(data_dir, supervised_label,w_bg=0.25)

# Autoencoder

In [17]:
class conv_block(nn.Module):
    def __init__(self,t_size,n_step):
        super(conv_block,self).__init__()
        self.cov1d_1 = nn.Conv2d(t_size,t_size,3,stride=1,padding=1,padding_mode = 'zeros')
        self.cov1d_2 = nn.Conv2d(t_size,t_size,3,stride=1,padding=1,padding_mode = 'zeros')
        self.cov1d_3 = nn.Conv2d(t_size,t_size,3,stride=1,padding=1,padding_mode = 'zeros')
        self.norm_3 = nn.LayerNorm(n_step)
        self.relu_1 = nn.ReLU()
        self.relu_2 = nn.ReLU()
        self.relu_3 = nn.ReLU()
#         self.relu_1 = nn.Tanh()
#         self.relu_2 = nn.Tanh()
#         self.relu_3 = nn.Tanh()
        #self.drop = nn.Dropout(p=0.2)
        
    def forward(self,x):
        x_input = x
        out = self.cov1d_1(x)
        out = self.relu_1(out)
        out = self.cov1d_2(out)
        out = self.relu_2(out)
        out = self.cov1d_3(out)
        out = self.norm_3(out)
        out = self.relu_3(out)
        #out = self.drop(out)
        out = out.add(x_input)
        
        return out

In [18]:
class identity_block(nn.Module):
    def __init__(self,t_size,n_step):
        super(identity_block,self).__init__()
        self.cov1d_1 = nn.Conv2d(t_size,t_size,3,stride=1,padding=1,padding_mode = 'zeros')
        self.norm_1 = nn.LayerNorm(n_step)
        #self.drop = nn.Dropout(p=0.2)
        self.relu = nn.ReLU()
#        self.relu = nn.Tanh()
        
    def forward(self,x):
        x_input = x
        out = self.cov1d_1(x)
        out = self.norm_1(out)
        out = self.relu(out)
        #output = self.drop(x)
        
        return out

# Rules to use the model architecture 
### Mask should be included since we use affine transformation function, so the "num_mask" should never be "None"
### The format of mask should be tensor list of the mask, so the fixed_mask should be the list of tenosr 

In [19]:
# narrow the range of the adjust parameter for the mask region, since it is not the noise free dataset,
# this will increase the background noise's influence to the MSE loss
# 
class Encoder(nn.Module):
    def __init__(self,original_step_size,pool_list,embedding_size,conv_size,device):
        super(Encoder,self).__init__()
        
        self.device = device
        blocks = []
        self.input_size_0 = original_step_size[0]
        self.input_size_1 = original_step_size[1]
        number_of_blocks = len(pool_list)
        blocks.append(conv_block(t_size=conv_size, n_step=original_step_size))
        blocks.append(identity_block(t_size=conv_size, n_step=original_step_size))
        blocks.append(nn.MaxPool2d(pool_list[0], stride=pool_list[0]))
        for i in range(1,number_of_blocks):
            original_step_size = [original_step_size[0]//pool_list[i-1],original_step_size[1]//pool_list[i-1]]
            blocks.append(conv_block(t_size=conv_size, n_step=original_step_size))
            blocks.append(identity_block(t_size=conv_size, n_step=original_step_size))
            blocks.append(nn.MaxPool2d(pool_list[i], stride=pool_list[i])) 
            
        self.block_layer = nn.ModuleList(blocks)
        self.layers=len(blocks)
        original_step_size = [original_step_size[0]//pool_list[-1],original_step_size[1]//pool_list[-1]]
        
        input_size = original_step_size[0]*original_step_size[1]
        self.cov2d = nn.Conv2d(1,conv_size,3,stride=1,padding=1,padding_mode = 'zeros')
        self.cov2d_1 = nn.Conv2d(conv_size,1,3,stride=1,padding=1,padding_mode = 'zeros')
        self.relu_1 = nn.ReLU()
        self.relu_2 = nn.ReLU()
        self.tanh = nn.Tanh()
        self.before = nn.Linear(input_size,20)
        self.embedding_size = embedding_size
    
        self.dense = nn.Linear(20,self.embedding_size)
    
    
    def forward(self,x,rotate_value = None):

        out = x.view(-1,1,self.input_size_0,self.input_size_1)
        out = self.cov2d(out)
        for i in range(self.layers):
            out = self.block_layer[i](out)
        out = self.cov2d_1(out)
        out = torch.flatten(out,start_dim=1)
        out = self.before(out) 
        out = self.dense(out)

        


        return out

In [20]:
device = torch.device('cuda')

In [ ]:
device

# Setting Parameters

## Parameters shared with both model architecture 

In [22]:
en_original_step_size=[200,200]
pool_list=[5,4,2]
de_original_step_size = [5,5]
up_list = [2,4,5]
embedding_size=4
conv_size =128

## Parameters for first Training Process

# Set Model Structure

# Use the generated rotation and scale shear to check on the base position and create new mask region

## Model for second process training

In [23]:
def make_model_2(device,
               en_original_step_size = en_original_step_size, 
               pool_list = pool_list, 
               de_original_step_size = de_original_step_size,
               up_list = up_list,
               embedding_size = embedding_size,
               conv_size = conv_size,
               learning_rate = 3e-5
               ):
    
    encoder = Encoder(original_step_size=en_original_step_size,
                      pool_list=pool_list,
                      embedding_size=embedding_size,
                      conv_size=conv_size,
                      device = device).to(device)
    
    optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
    
    return encoder, optimizer

In [24]:
encoder, optimizer = make_model_2(torch.device('cpu'))

In [ ]:
sum(p.numel() for p in encoder.parameters() if p.requires_grad)

# Loss Function for Second Process
## loss function used for small batch size

## loss function used for large batch size

In [26]:
def loss_function_2nd(join,
                      train_iterator,
                      optimizer,
                      device,
                     ):


    # set the train mode
    join.train()

    # loss of the epoch
    train_loss = 0
    
    for x,label in tqdm(train_iterator, leave=True, total=len(train_iterator)):
     

        x = x.to(device, dtype=torch.float)
        label = label.to(device, dtype=torch.float)

        # update the gradients to zero
        optimizer.zero_grad()
        

        out= join(x)
        
        
        loss  = F.mse_loss(out, label, reduction='mean')


        if loss>1.5:

            loss = F.l1_loss(out, label, reduction='mean')


        if loss>2:
            loss=2

                
        # backward pass
        train_loss += loss.item()
        loss.backward()
        # update the weights
        optimizer.step()

        return train_loss

# Second training process
### set training parameters

## RayTune Training Process

In [27]:
data_dir = os.path.abspath("./polycrystal_output4D.mat")
folder_path = os.path.abspath("test_loss_function")
#pretrain_weight = os.path.abspath("./04_20_RAYTUNE_lr:0.000065_scale_cof:80.350_shear_cof:16.100_MAE:0.0063_seed:42_epoch:0004_trainloss:0.002384_l1:0.00014_scal:0.00000_shr:0.00000.pkl")

In [28]:
#pretrain_weight

In [ ]:
folder_path

Define the loss landscapes with Hessian

In [30]:
import copy
from pyhessian import hessian

def get_params(model_orig,  model_perb, direction, alpha):
    for m_orig, m_perb, d in zip(model_orig.parameters(), model_perb.parameters(), direction):
        m_perb.data = m_orig.data + alpha * d
    return model_perb

def set_params(model_perb, weights_perb):
    for m_perb, w_perb in zip(model_perb.parameters(), weights_perb):
        m_perb.data = w_perb
    return model_perb

def get_loss_landscapes(join, train_iterator, optimizer, device, STEPS = 21, DIM = 2, POINTS = 441, START = -2.0, END = 2.0):
    
    # set the train mode
    join.train()

    for x,label in tqdm(train_iterator, leave=True, total=len(train_iterator)):
     

        x = x.to(device, dtype=torch.float)
        label = label.to(device, dtype=torch.float)

        # update the gradients to zero
        optimizer.zero_grad()


        model_perb = copy.deepcopy(join)
        model_perb.eval()
        model_current = copy.deepcopy(join)
        model_current.eval()

        # make sure the numbers of sampling points is less than the total number of points
        FULL_CUBE = False
        if POINTS >= STEPS ** DIM:
            POINTS = STEPS ** DIM
            FULL_CUBE = True
            print("Full cube calculation. Total points: ", POINTS)
        else:
            print("Random sampling cube calculation. Total points: ", POINTS)
        

        # Generate the loss values array using BFS
        # Create a coordinate array for loss values
        loss_coordinates_list = []
        pbar = tqdm(total=POINTS, desc="Generating 2-D coordinates in the subspace")
        for i in range(STEPS):
            for j in range(STEPS):
                t = (i,j)
                loss_coordinates_list.append(t)
                pbar.update(1)
        pbar.close()
        loss_coordinates = np.array(loss_coordinates_list)

        # print(loss_coordinates)
        print(loss_coordinates.shape)

        # Create a data matrix to store loss values
        data_matrix = np.empty([POINTS, 1], dtype=float)
        # Fill array with initial value (e.g., -1)
        data_matrix.fill(-1)
        # print(data_matrix)
        print(data_matrix.shape)

        criterion = F.mse_loss

        # calculate the hessian eigenvalues and eigenvectors
        FLAG = True if torch.cuda.is_available() else False
        print("start creating the hessian computation module")
        hessian_comp = hessian(join, criterion, data=(x, label), cuda=FLAG)
        print("finish creating the hessian computation module and start calculating the trace")
        trace = hessian_comp.trace()
        print("The trace of this model is: %.4f"%(np.mean(trace)))
        print("finish calculating the hessian trace and start calculating the eigenvecotrs")
        top_eigenvalues, top_eigenvector = hessian_comp.eigenvalues(top_n=DIM)
        print("finish calculating the eigenvecotrs")
        lams = np.linspace(START, END, STEPS).astype(np.float32)
        
        # calculate the hessian loss values
        for j in tqdm(range(POINTS), desc="Calculating sampling loss values in the subspace"):
            # adjust the model and fill with a loss with corresponding model parameters
            next_pos = tuple(loss_coordinates[j])
            model_current = copy.deepcopy(join)
            for i in range(DIM):
                model_perb = get_params(model_current, model_perb, top_eigenvector[i], lams[next_pos[i]])
                model_current = copy.deepcopy(model_perb)
            # calculate the loss value
            this_loss = criterion(model_current(x), label, reduction='mean').item()
            if this_loss > 1.5:
                this_loss = F.l1_loss(model_current(x), label, reduction='mean').item()
            if this_loss > 2:
                this_loss = 2
        
            data_matrix[j] = this_loss
        
        # save the loss values
        np.save(folder_path + '/supervised_loss_values.npy', data_matrix)
        np.save(folder_path + '/supervised_loss_coordinates.npy', loss_coordinates)
        print("Loss values saved.")
        # plot the loss values
        X, Y = np.meshgrid(np.linspace(START, END, STEPS), np.linspace(START, END, STEPS))
        Z = data_matrix.reshape(STEPS, STEPS)
        fig, ax = plt.subplots()
        ax.contour(X, Y, Z, levels=80)
        plt.title("Loss landscape for supervised learning of 4DSTEM")
        plt.savefig(folder_path + '/supervised_loss_landscape.png')

# Use Cross Validation and check with training 

In [31]:
dataloader = DataLoader(data_, batch_size=64, shuffle=True, num_workers=0)

In [32]:
def train_function( 
                 dataloader,
                 epochs=500,
                 learning_rate = 3e-5,
                 max_rate = 2e-4,
                 up_size = 100,
                 epoch_ = None,
                 file_path = None,
                 folder_path=folder_path,
                 best_train_loss= None,
                 set_scheduler = False,

                ):
        
            
        device = "cpu"
        if torch.cuda.is_available():
            device = "cuda"
        
        
        seed = 42
        np.random.seed(seed)
        torch.manual_seed(seed)
        print('.........a step.........')
        
        patience = 0
        
        print("........successfully load parameters")
        
        encoder,optimizer = \
        make_model_2(device,learning_rate = learning_rate)
        
        radius = 45
        
            
        if set_scheduler:
        
            lr_scheduler = torch.optim.lr_scheduler.CyclicLR(optimizer, base_lr=learning_rate, max_lr=max_rate,
                                                  step_size_up=up_size,cycle_momentum=False)
        else: 
            
            lr_scheduler = None
        
            
     
        print('..........successfully generate model')
        

        N_EPOCHS = epochs

        if best_train_loss == None:
            best_train_loss = float('inf')


        if epoch_==None:
            start_epoch = 0
        else:
            start_epoch = epoch_+1
        print('...........successfully generate train interator')

        for epoch in range(start_epoch,epochs):
    
            train = loss_function_2nd(encoder,dataloader, optimizer,device)

            input_length = len(dataloader)
                
                
            train_loss = train
            
            train_loss /= input_length

    #        VAE_L /= len(train_iterator)
            print(f'Epoch {epoch}, Train Loss: {train_loss:.4f}')
    #        print(f'......... VAE Loss: {VAE_L:.4f}')
            print('.............................')
            
            # generate the loss landscape for this epoch
            get_loss_landscapes(encoder,dataloader, optimizer,device)
            
            checkpoint = {
                "encoder":encoder.state_dict(),
                'optimizer': optimizer.state_dict(),
                "epoch": epoch,
                'mse_loss': train_loss,
            }
            if best_train_loss > train_loss:
                best_train_loss = train_loss
                lr_ = format(optimizer.param_groups[0]['lr'],'.6f')
                file_path = folder_path+'/_lr:'+lr_+\
                           f'_epoch:{epoch:04d}_trainloss:{train_loss:.6f}_.pkl'
                torch.save(checkpoint, file_path)
                

            if lr_scheduler!= None:
                lr_scheduler.step()


In [ ]:
train_function(dataloader)